In [74]:
import numpy as np
import pandas as pd
from statistics import mean
import math
from scipy.optimize import minimize

In [2]:
ls

GMM & MLE.ipynb  hs_ml.jl         hsdata.csv


In [3]:
dataset = pd.read_csv("hsdata.csv")

In [54]:
consumption = [math.exp(i) for i in list(dataset['c'])]
asset_return = [math.exp(i) for i in list(dataset['r'])]
dividend = [math.exp(i) for i in list(dataset['d'])]
dataset['consumption'] = consumption
dataset['asset_return'] = asset_return 

# (a)

## (1) one step GMM

Roadmap </br>
1. construct a sample analogy (3X1 vector) for the moment equation which depends on $$\{\delta_0,\gamma_0\}$$
2. construct the GMM criterion function $$ J(\delta, \gamma) = n\bar{g_n}(\delta, \gamma)'W\bar{g_n}(\delta, \gamma)$$
3. using gradient descent, find the minimizer $$\{\delta_{gmm},\gamma_{gmm}\}$$

In [36]:
# 0. initial value 
delta = np.random.uniform(low=0.1, high=0.99)
gamma = np.random.uniform(low=0.1, high=0.99)

In [108]:
sample = np.array([0.22760131, 0.71847686])

In [59]:
range(1,time)

range(1, 250)

In [70]:
def rosen(parameter):
    delta = parameter[0]
    gamma = parameter[1]
    # 1. construct a sample analogy
    time = len(dataset['time'])
    sample_analogy_1 = []
    for i in range(time-1):
        c_tmr = list(dataset['consumption'])[i+1]
        c_t = list(dataset['consumption'])[i]
        R_tmr = dataset['asset_return'][i]
        sample_analogy_1.append(delta*((c_tmr/c_t)**(-gamma))*R_tmr -1)

    sample_analogy_2 = []
    for i in range(1, time-1):
        c_tmr = list(dataset['consumption'])[i+1]
        c_t = list(dataset['consumption'])[i]
        R_tmr = dataset['asset_return'][i]
        R_t = dataset['asset_return'][i-1]
        sample_analogy_2.append((delta*((c_tmr/c_t)**(-gamma))*R_tmr -1)*R_t)

    sample_analogy_3 = []
    for i in range(1, time-1):
        c_tmr = list(dataset['consumption'])[i+1]
        c_ytd = dataset['consumption'][i-1]
        c_t = list(dataset['consumption'])[i]
        R_tmr = dataset['asset_return'][i]
        sample_analogy_3.append((delta*((c_tmr/c_t)**(-gamma))*R_tmr -1)*(c_t/c_ytd))

    # 2. Calculate GMM Criterion Function
    g_n_bar = np.array([mean(sample_analogy_1), mean(sample_analogy_2), mean(sample_analogy_3)])
    criterion = g_n_bar @ g_n_bar
    
    return criterion

In [75]:
res = minimize(rosen, sample, method='nelder-mead',
               options={'xatol': 1e-8, 'disp': True})

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 97
         Function evaluations: 186


In [76]:
res.x

array([ 0.92071863, -2.16647848])

$$Hence \ under \ \hat{W}=I \ we\ have, \\  \delta_{gmm} = 0.92071863, \\  \gamma_{gmm} = -2.16647848  $$

## (2) two-step GMM

Roadmap </br>
1. estimate W, using the result from (1)
2. plug the estimated W into the GMM criterion, and then do the GMM estimation 

In [104]:
#0 set the parameter
delta = 0.92071863
gamma = -2.16647848

#1 calculate the residual using the result in (1)
variance_sample_analogy = []
sample_number = 0
for i in range(1, time-1):
        c_tmr = list(dataset['consumption'])[i+1]
        c_ytd = dataset['consumption'][i-1]
        c_t = list(dataset['consumption'])[i]
        R_tmr = dataset['asset_return'][i]
        e_hat = delta*((c_tmr/c_t)**(-gamma))*R_tmr -1
        
        Z1 = 1
        Z2 = dataset['asset_return'][i-1]
        Z3 = c_t/c_ytd
        Z = np.array([Z1, Z2, Z3])
        ZZ = np.outer(Z, Z)
        
        variance_sample_analogy.append((e_hat**2)*ZZ)
        sample_number += 1

omega = 0
for var in variance_sample_analogy:
    omega = omega + var
omega = (1/sample_number)*omega

W_hat = np.linalg.inv(omega)

In [109]:
def rosen2(parameter):
    delta = parameter[0]
    gamma = parameter[1]
    # 1. construct a sample analogy
    time = len(dataset['time'])
    sample_analogy_1 = []
    for i in range(time-1):
        c_tmr = list(dataset['consumption'])[i+1]
        c_t = list(dataset['consumption'])[i]
        R_tmr = dataset['asset_return'][i]
        sample_analogy_1.append(delta*((c_tmr/c_t)**(-gamma))*R_tmr -1)

    sample_analogy_2 = []
    for i in range(1, time-1):
        c_tmr = list(dataset['consumption'])[i+1]
        c_t = list(dataset['consumption'])[i]
        R_tmr = dataset['asset_return'][i]
        R_t = dataset['asset_return'][i-1]
        sample_analogy_2.append((delta*((c_tmr/c_t)**(-gamma))*R_tmr -1)*R_t)

    sample_analogy_3 = []
    for i in range(1, time-1):
        c_tmr = list(dataset['consumption'])[i+1]
        c_ytd = dataset['consumption'][i-1]
        c_t = list(dataset['consumption'])[i]
        R_tmr = dataset['asset_return'][i]
        sample_analogy_3.append((delta*((c_tmr/c_t)**(-gamma))*R_tmr -1)*(c_t/c_ytd))

    # 2. Calculate GMM Criterion Function
    g_n_bar = np.array([mean(sample_analogy_1), mean(sample_analogy_2), mean(sample_analogy_3)])
    criterion = g_n_bar @ W_hat @ g_n_bar
    
    return criterion

In [118]:
res2 = minimize(rosen2, sample, method='nelder-mead',
               options={'xatol': 1e-8, 'disp': True})

Optimization terminated successfully.
         Current function value: 0.000767
         Iterations: 89
         Function evaluations: 174


In [119]:
res2.x

array([ 0.92052058, -2.40453142])

$$Hence\ under\ two-step\ GMM \ we\ have, \\  \delta_{gmm2} = 0.92052058, \\  \gamma_{gmm2} = -2.40453142  $$

## (3) 

## (3-1) repeat part (1)

In [121]:
def rosen3(parameter):
    delta = parameter[0]
    gamma = parameter[1]
    # 1. construct a sample analogy
    time = len(dataset['time'])
    sample_analogy_1 = []
    for i in range(time-1):
        c_tmr = list(dataset['consumption'])[i+1]
        c_t = list(dataset['consumption'])[i]
        R_tmr = dataset['asset_return'][i]
        sample_analogy_1.append(delta*((c_tmr/c_t)**(-gamma))*R_tmr -1)

    sample_analogy_2 = []
    for i in range(1, time-1):
        c_tmr = list(dataset['consumption'])[i+1]
        c_t = list(dataset['consumption'])[i]
        R_tmr = dataset['asset_return'][i]
        R_t = dataset['asset_return'][i-1]
        sample_analogy_2.append((delta*((c_tmr/c_t)**(-gamma))*R_tmr -1)*R_t)

    sample_analogy_3 = []
    for i in range(1, time-1):
        c_tmr = list(dataset['consumption'])[i+1]
        c_ytd = dataset['consumption'][i-1]
        c_t = list(dataset['consumption'])[i]
        R_tmr = dataset['asset_return'][i]
        sample_analogy_3.append((delta*((c_tmr/c_t)**(-gamma))*R_tmr -1)*(c_t/c_ytd))
        
    sample_analogy_4 = []
    for i in range(2, time-1):
        c_tmr = list(dataset['consumption'])[i+1]
        c_ytd = dataset['consumption'][i-1]
        c_t = list(dataset['consumption'])[i]
        R_tmr = dataset['asset_return'][i]
        R_ytd = dataset['asset_return'][i-2]
        sample_analogy_4.append((delta*((c_tmr/c_t)**(-gamma))*R_tmr -1)*(R_ytd))

    sample_analogy_5 = []
    for i in range(2, time-1):
        c_tmr = list(dataset['consumption'])[i+1]
        c_ytd = dataset['consumption'][i-1]
        c_ytd2 = dataset['consumption'][i-2]
        c_t = list(dataset['consumption'])[i]
        R_tmr = dataset['asset_return'][i]
        sample_analogy_5.append((delta*((c_tmr/c_t)**(-gamma))*R_tmr -1)*(c_ytd/c_ytd2))


    # 2. Calculate GMM Criterion Function
    g_n_bar = np.array([mean(sample_analogy_1), mean(sample_analogy_2), mean(sample_analogy_3), 
                        mean(sample_analogy_4), mean(sample_analogy_5)])
    criterion = g_n_bar @ g_n_bar
    
    return criterion

In [122]:
res3 = minimize(rosen3, sample, method='nelder-mead',
               options={'xatol': 1e-8, 'disp': True})

Optimization terminated successfully.
         Current function value: 0.000001
         Iterations: 106
         Function evaluations: 203


In [124]:
res3.x

array([ 0.92037327, -2.19448185])

$$Hence\ under\ this\ specification\ we\ have, \\  \delta_{gmm2} = 0.92037327, \\  \gamma_{gmm2} = -2.19448185  $$

## (3-2) repeat part (2)

In [125]:
#0 set the parameter
delta = 0.92037327
gamma = -2.19448185

#1 calculate the residual using the result in (1)
variance_sample_analogy2 = []
sample_number = 0
for i in range(2, time-1):
        c_tmr = list(dataset['consumption'])[i+1]
        c_ytd = dataset['consumption'][i-1]
        c_ytd2 = dataset['consumption'][i-2]
        c_t = list(dataset['consumption'])[i]
        R_ytd = dataset['asset_return'][i-2]
        R_tmr = dataset['asset_return'][i]
        e_hat = delta*((c_tmr/c_t)**(-gamma))*R_tmr -1
        
        
        Z1 = 1
        Z2 = dataset['asset_return'][i-1]
        Z3 = c_t/c_ytd
        Z4 = R_ytd
        Z5 = c_ytd/c_ytd2
        Z = np.array([Z1, Z2, Z3, Z4, Z5])
        ZZ = np.outer(Z, Z)
        
        variance_sample_analogy2.append((e_hat**2)*ZZ)
        sample_number += 1

omega2 = 0
for var in variance_sample_analogy2:
    omega2 = omega2 + var
omega2 = (1/sample_number)*omega2

W_hat2 = np.linalg.inv(omega2)

In [126]:
def rosen4(parameter):
    delta = parameter[0]
    gamma = parameter[1]
    # 1. construct a sample analogy
    time = len(dataset['time'])
    sample_analogy_1 = []
    for i in range(time-1):
        c_tmr = list(dataset['consumption'])[i+1]
        c_t = list(dataset['consumption'])[i]
        R_tmr = dataset['asset_return'][i]
        sample_analogy_1.append(delta*((c_tmr/c_t)**(-gamma))*R_tmr -1)

    sample_analogy_2 = []
    for i in range(1, time-1):
        c_tmr = list(dataset['consumption'])[i+1]
        c_t = list(dataset['consumption'])[i]
        R_tmr = dataset['asset_return'][i]
        R_t = dataset['asset_return'][i-1]
        sample_analogy_2.append((delta*((c_tmr/c_t)**(-gamma))*R_tmr -1)*R_t)

    sample_analogy_3 = []
    for i in range(1, time-1):
        c_tmr = list(dataset['consumption'])[i+1]
        c_ytd = dataset['consumption'][i-1]
        c_t = list(dataset['consumption'])[i]
        R_tmr = dataset['asset_return'][i]
        sample_analogy_3.append((delta*((c_tmr/c_t)**(-gamma))*R_tmr -1)*(c_t/c_ytd))
        
    sample_analogy_4 = []
    for i in range(2, time-1):
        c_tmr = list(dataset['consumption'])[i+1]
        c_ytd = dataset['consumption'][i-1]
        c_t = list(dataset['consumption'])[i]
        R_tmr = dataset['asset_return'][i]
        R_ytd = dataset['asset_return'][i-2]
        sample_analogy_4.append((delta*((c_tmr/c_t)**(-gamma))*R_tmr -1)*(R_ytd))

    sample_analogy_5 = []
    for i in range(2, time-1):
        c_tmr = list(dataset['consumption'])[i+1]
        c_ytd = dataset['consumption'][i-1]
        c_ytd2 = dataset['consumption'][i-2]
        c_t = list(dataset['consumption'])[i]
        R_tmr = dataset['asset_return'][i]
        sample_analogy_5.append((delta*((c_tmr/c_t)**(-gamma))*R_tmr -1)*(c_ytd/c_ytd2))


    # 2. Calculate GMM Criterion Function
    g_n_bar = np.array([mean(sample_analogy_1), mean(sample_analogy_2), mean(sample_analogy_3), 
                        mean(sample_analogy_4), mean(sample_analogy_5)])

    criterion = g_n_bar @ W_hat2 @ g_n_bar
    
    return criterion

In [127]:
res4 = minimize(rosen4, sample, method='nelder-mead',
               options={'xatol': 1e-8, 'disp': True})

Optimization terminated successfully.
         Current function value: 0.024855
         Iterations: 87
         Function evaluations: 170


In [128]:
res4.x

array([ 0.92040723, -2.52268154])

$$Hence\ under\ this\ specification\ we\ have, \\  \delta_{gmm2} = 0.92040723, \\  \gamma_{gmm2} = -2.52268154  $$